In [1]:
# Add Lincoln to system path
import sys
sys.path.append("/Users/seth/development/lincoln/")

In [2]:
from torch import Tensor
import torch
import numpy as np
from numpy import random

import typing
from typing import List, Tuple

# Demo

Building a simple classifier on the breast cancer dataset.

## Data prep

In [3]:
from sklearn.datasets import load_breast_cancer
breast_cancer = load_breast_cancer()
features = breast_cancer.data
labels = breast_cancer.target
feature_names = breast_cancer.feature_names

/Users/seth/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/Users/seth/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


In [4]:
from lincoln.utils import standardize

features = standardize(features)

In [5]:
import lincoln as lnc
from lincoln.layers import Dense, NeuralNetwork
from lincoln.losses import LogSigmoidLoss, MeanSquaredError, LogLoss, Loss
from lincoln.optimizers import SGD
from lincoln.layers import Layer
from lincoln.activations import Sigmoid, LogSigmoid
from lincoln.models import Logistic

### `Logistic` model 

In [6]:
# network = NeuralNetwork([
#     Dense(1, activation=Sigmoid()),
# ])

# model = Logistic(network,
#                  loss=MeanSquaredError(), 
#                  optimizer=SGD(0.1))

# model.fit(features, labels, batch_size=128, log_ps=False,
#           epochs=100, print_every=10)

Something is off...

In [7]:
# network = lnc.Sequential(
#             lnc.layers.Dense(10),
#             lnc.layers.Dense(1, activation=LogSigmoid()),
#             )
# model = Logistic(network,
#                  loss=LogSigmoidLoss(network), 
#                  optimizer=SGD(0.003))
# model.fit(features, labels, batch_size=128, log_ps=True,
#           epochs=500, print_every=100)

**TODO:** `print_every` not working.

## Regression network

In [8]:
from sklearn.datasets import load_boston

boston = load_boston()

data = boston.data
target = boston.target
features = boston.feature_names

from sklearn.preprocessing import StandardScaler
s = StandardScaler()
data = s.fit_transform(data)

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(data, target, test_size=0.3, random_state=80718)

X_train, X_test, y_train, y_test = Tensor(X_train), Tensor(X_test), Tensor(y_train), Tensor(y_test)

/Users/seth/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


In [9]:
X_train

tensor([[-0.4104, -0.4877, -1.3069,  ...,  0.1130,  0.4411, -1.0265],
        [ 5.5357, -0.4877,  1.0160,  ...,  0.8066, -3.8822, -0.3565],
        [ 9.9417, -0.4877,  1.0160,  ...,  0.8066,  0.4411,  0.6388],
        ...,
        [-0.3451, -0.4877, -0.4373,  ...,  1.1765,  0.4411, -0.6158],
        [-0.4133,  0.5853, -0.9158,  ...,  0.2517,  0.4271, -0.7616],
        [-0.3034, -0.4877, -0.4373,  ...,  1.1765,  0.4152,  1.0130]])

In [10]:
X_test

tensor([[-0.4144,  3.5896, -1.2339,  ..., -1.7364,  0.3959, -1.2396],
        [ 1.6619, -0.4877,  1.0160,  ...,  0.8066,  0.4064,  1.2794],
        [ 0.2151, -0.4877,  1.0160,  ...,  0.8066, -0.0152,  0.7117],
        ...,
        [-0.3880, -0.4877, -0.1805,  ..., -0.0257,  0.4345, -0.1308],
        [-0.4139,  1.7656, -0.8487,  ..., -0.8579,  0.4411, -0.7728],
        [-0.4161,  3.5896, -1.2339,  ..., -1.7364,  0.3714, -1.3699]])

## Custom `Model` class and `train` function

In [11]:
from lincoln.losses import MeanSquaredError
from lincoln.optimizers import SGD
from lincoln.utils import permute_data, generate_batch, to_2d

from lincoln.activations import LinearAct

In [16]:
class LinearModel(object):
    def __init__(self, 
                 network: NeuralNetwork):
        self.network = network

    def train(self, 
              X_train: Tensor, 
              y_train: Tensor,
              X_test: Tensor = None,
              y_test: Tensor = None,
              iterations: int=500, print_every: int=100, 
              batch_size: int=32, seed: int = 1)-> None:
 
        torch.manual_seed(seed)
        y_train, y_test = to_2d(y_train, "col"), to_2d(y_test, "col")
        start = 0

        for i in range(iterations):
                 
            # Generate batch
            if start >= X_train.shape[0]:
                X_train, y_train = permute_data(X_train, y_train)
                start = 0

            X_batch, y_batch = generate_batch(X_train, 
                                              y_train, 
                                              start, batch_size)

            self.network.train_batch(X_batch, y_batch)
            
            if i % print_every == 0:x
                loss = self.network.forward_loss(X_test, y_test)
                print("Loss: ", loss) 

In [19]:
lr = NeuralNetwork(
    layers=[Dense(neurons=1, 
                  activation=LinearAct())], 
            learning_rate=0.0002,
            loss = MeanSquaredError())


model = LinearModel(network=lr)

In [20]:
num_iter = 10000
print_every = 100

model.train(X_train, y_train, X_test, y_test, 
      iterations = num_iter, 
      print_every = print_every, 
      batch_size = 23, 
      seed=80718);

Loss:  tensor(82850.5078)
Loss:  tensor(186947.)
Loss:  tensor(889866.)
Loss:  tensor(2191602.5000)
Loss:  tensor(4092166.7500)
Loss:  tensor(6591570.)
Loss:  tensor(9689808.)
Loss:  tensor(13386880.)
Loss:  tensor(17682798.)
Loss:  tensor(22577560.)
Loss:  tensor(28071134.)
Loss:  tensor(34163548.)
Loss:  tensor(40854828.)
Loss:  tensor(48144984.)
Loss:  tensor(56033948.)
Loss:  tensor(64521732.)
Loss:  tensor(73608304.)
Loss:  tensor(83293648.)
Loss:  tensor(93577808.)
Loss:  tensor(104460800.)
Loss:  tensor(115942592.)
Loss:  tensor(128023192.)
Loss:  tensor(140702608.)
Loss:  tensor(153980864.)
Loss:  tensor(167857968.)
Loss:  tensor(182333760.)
Loss:  tensor(197408528.)
Loss:  tensor(213082160.)
Loss:  tensor(229354544.)
Loss:  tensor(246225856.)
Loss:  tensor(263695904.)
Loss:  tensor(281765024.)
Loss:  tensor(300433280.)
Loss:  tensor(319700384.)
Loss:  tensor(339566304.)
Loss:  tensor(360031104.)
Loss:  tensor(381094784.)
Loss:  tensor(402757312.)
Loss:  tensor(425018720.)
Loss

In [ ]:
lr = NeuralNetwork(
    layers=[Dense(neurons=1, 
                   activation=LinearAct())],
)
loss = MeanSquaredError()
optim = SGD(lr=0.0002)

model = LinearModel(network=lr, 
              loss=MeanSquaredError(), 
              optimizer=optim)

In [ ]:
num_iter = 1000
print_every = 100

model.train(X_train, y_train, X_test, y_test, 
      iterations = num_iter, 
      print_every = print_every, 
      batch_size = 23, 
      seed=80718);

In [ ]:
model.network.layers[0].params